In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import os


data_dir = "../dataset/n"


data_gen = ImageDataGenerator(rescale = 1./255,validation_split =0.2)


#dados pra treino
train_generator =data_gen.flow_from_directory(
    data_dir,
    target_size=(28,28),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

#dados pra validação
validation_generator = data_gen.flow_from_directory(
    data_dir,
    target_size=(28, 28),
    batch_size=32,
    class_mode='categorical',  
    subset='validation'
)


Found 6713 images belonging to 15 classes.
Found 1671 images belonging to 15 classes.


In [2]:
model = Sequential()

#camadas da rede

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))


model.add(Dense(15, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


model.summary()

c:\Users\rafae\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │         1,935 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 226,255 (883.81 KB)

 Trainable params: 226,255 (883.81 KB)

 Non-trainable params: 0 (0.00 B)

In [3]:
#treinamento da rede
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min', restore_best_weights=True)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=50,
    callbacks=[early_stopping]
)

#avaliação da rede
loss, accuracy = model.evaluate(validation_generator)
print(f'Acurácia do modelo: {accuracy * 100:.2f}%')







c:\Users\rafae\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
209/209 ━━━━━━━━━━━━━━━━━━━━ 15s 64ms/step - accuracy: 0.2008 - loss: 2.4221 - val_accuracy: 0.7308 - val_loss: 1.0016
Epoch 2/50
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 273us/step - accuracy: 0.6562 - loss: 1.1545 - val_accuracy: 0.8571 - val_loss: 0.8114
Epoch 3/50
  3/209 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.5903 - loss: 1.3527 

c:\Users\rafae\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


209/209 ━━━━━━━━━━━━━━━━━━━━ 7s 33ms/step - accuracy: 0.6430 - loss: 1.1208 - val_accuracy: 0.8035 - val_loss: 0.6564
Epoch 4/50
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 97us/step - accuracy: 0.7500 - loss: 0.7714 - val_accuracy: 0.5714 - val_loss: 0.8965
Epoch 5/50
209/209 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - accuracy: 0.7513 - loss: 0.7913 - val_accuracy: 0.8600 - val_loss: 0.4833
Epoch 6/50
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 97us/step - accuracy: 0.8438 - loss: 0.5822 - val_accuracy: 1.0000 - val_loss: 0.4528
Epoch 7/50
209/209 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - accuracy: 0.8038 - loss: 0.6034 - val_accuracy: 0.8750 - val_loss: 0.3996
Epoch 8/50
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 89us/step - accuracy: 0.7188 - loss: 0.8539 - val_accuracy: 0.5714 - val_loss: 1.2412
Epoch 9/50
209/209 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - accuracy: 0.8221 - loss: 0.5264 - val_accuracy: 0.9002 - val_loss: 0.3573
Epoch 10/50
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 100us/step - accuracy: 0.8750 - loss: 0.4299 - val_accuracy: 0.8

In [6]:

img_dir = '../test/'

# caminho das imagens
img_paths = [os.path.join(img_dir, img_name) for img_name in os.listdir(img_dir)] 


#array para guardar valores preditos 
predictions = []

#carregar e prever valores da imagens
for img_path in img_paths:
  
    img = image.load_img(img_path, target_size=(28, 28))
    
  
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  
    img_array /= 255.0 

   
    prediction = model.predict(img_array)
    
   
    predicted_class = np.argmax(prediction)
    
   
    predictions.append(predicted_class)



for i, img_path in enumerate(img_paths):
    print(f"Imagem: {img_path}, Classe prevista: {predictions[i]}")




n = -1
i = 0
n1 = ''
n2 = ''

#loop para pegar dígitos até chegar no símbolo de operação
while n <=10:
    n1 = n1 + str(predictions[i])
    n = predictions[i+1]
    i +=1


i +=1
#loop para pegar dígitos depois do símbolo de operação
for j in range(i, len(predictions)):  
    n2 += str(predictions[j])
    if predictions[j] >= 10:
        n = -1  
        break 


n1 = int(n1)
n2 = int(n2)
op = n



#dictionary para definir operação a ser feita 
operations = {
    10: lambda a, b: a + b,
    11: lambda a, b: int(a / b) if b != 0 else "Divisão por zero",
    12: lambda a, b: a == b,
    13: lambda a, b: a * b,
    14: lambda a, b: a - b
}
   
n3 = operations.get(op, lambda a, b: "Operação inválida")(n1, n2)


print(f'resultado: {n3}' )




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Imagem: ../test/a1.png, Classe prevista: 5
Imagem: ../test/a2.png, Classe prevista: 8
Imagem: ../test/a3.png, Classe prevista: 9
Imagem: ../test/b.png, Classe prevista: 14
Imagem: ../test/c1.png, Classe prevista: 3
Imagem: ../test/c2.png, Classe prevista: 7
resultado: 552
